# CodeLlama 7B - Instructor for code generation

In [1]:
!pip install git+https://github.com/huggingface/transformers.git@main accelerate bitsandbytes

  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-bl_q3i3k
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-bl_q3i3k
  Resolved https://github.com/huggingface/transformers.git to commit de13a951b38b85195984164819f1ab05fe508677
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.1 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.38.0.dev0-py3-none-any.whl size=8422159 sha256=9c62dda56e42ba759230221d712d2c56b08c75cbd22bce79e62a7c15e81a4d2d
  Stored in directory: /tmp/pip-ephem-wheel-cache-hg257m15/wheels/cf/59/82/6492402e887a68975030bf8c06532260abc16abb7ccd8127cc
Successfully built transformers
  Attempting uninstall: 

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

## **1.** Model

In [3]:
def setup(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype = "auto",
        load_in_4bit = True,
        trust_remote_code = True)
    return tokenizer, model

In [4]:
tokenizer, model = setup("codellama/CodeLlama-7b-Instruct-hf")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

# **2.** Input

In [5]:
def generate_code(text, tokenizer, model):
    inputs = tokenizer(text, return_tensors="pt").to(model.device)
    tokens = model.generate(
        **inputs,
        max_new_tokens = 128,
        temperature = 0.1,
        top_p = 0.9,
        do_sample = True,
        pad_token_id = tokenizer.eos_token_id,
        eos_token_id = tokenizer.eos_token_id,
        num_return_sequences = 1)
    return tokenizer.decode(tokens[0], skip_special_tokens=True)

# **3.** Inference

### **3.1.** Python

In [6]:
%%time
print(generate_code("# python\ndef delete_last_word_in_string(", tokenizer, model))

# python
def delete_last_word_in_string(string):
    return string[:-1]


def delete_last_word_in_string_2(string):
    return string[:-len(string.split()[-1])]


def delete_last_word_in_string_3(string):
    return string[:string.rfind(' ')]


def delete_last_word_in_string_4(string):
    return string[:string.rfind(' ')]


def delete_last_word_in_string_5(string):
    return string[:string
CPU times: user 13.8 s, sys: 786 ms, total: 14.5 s
Wall time: 20.7 s


In [8]:
%%time

text = '''\
def delete_last_word_in_sentence(sentence: str):
    """
    In Python, given a string of words, return the same string without the last word.
    """\
'''

print(generate_code(text, tokenizer, model))

def delete_last_word_in_sentence(sentence: str):
    """
    In Python, given a string of words, return the same string without the last word.
    """
    return sentence.split(" ")[0:-1]


def delete_last_word_in_sentence_2(sentence: str):
    """
    In Python, given a string of words, return the same string without the last word.
    """
    return sentence.split(" ")[0:-1]


def delete_last_word_in_sentence_3(sentence: str):
    """
    In Python, given a string of words, return the same string without the last word.
    """
    return sentence.rs
CPU times: user 12.1 s, sys: 106 ms, total: 12.2 s
Wall time: 17.4 s


In [10]:
# Run the generated function

def delete_last_word_in_sentence(sentence: str):
    """
    In Python, given a string of words, return the same string without the last word.
    """
    return sentence.split(" ")[0:-1]

delete_last_word_in_sentence("lets test this sentence")

['lets', 'test', 'this']

In [12]:
# Test human language rather than code completion

%%time
print(generate_code("Write a python function to compute the pi value with monte carlo method", tokenizer, model))

Write a python function to compute the pi value with monte carlo method. The function should take the number of iterations as input and return the pi value.

\begin{code}
def monte_carlo(n):
    x = np.random.uniform(0,1,n)
    y = np.random.uniform(0,1,n)
    inside = np.where(x**2 + y**2 <= 1, True, False)
    pi_value = 4*np.mean(inside)
    return pi_value
\end{code}

Comment: What is your question?

Comment
CPU times: user 11.1 s, sys: 121 ms, total: 11.2 s
Wall time: 13.4 s


In [14]:
# Run the generated function

import numpy as np

def monte_carlo(n):
    x = np.random.uniform(0,1,n)
    y = np.random.uniform(0,1,n)
    inside = np.where(x**2 + y**2 <= 1, True, False)
    pi_value = 4*np.mean(inside)
    return pi_value

monte_carlo(100000)

3.1512

### **3.2.** C++

In [15]:
%%time
print(generate_code("Write a C++ function for computing the n-th Fibonacci number", tokenizer, model))

Write a C++ function for computing the n-th Fibonacci number. The function should take an integer n as input and return the n-th Fibonacci number as output.

The Fibonacci sequence is defined as follows:

F(0) = 0
F(1) = 1
F(n) = F(n-1) + F(n-2) for n >= 2

For example, F(5) = 5, F(6) = 8, F(7) = 13, F(8) = 21, F(9) = 34, F(
CPU times: user 12.3 s, sys: 119 ms, total: 12.5 s
Wall time: 15.7 s


In [16]:
%%time
print(generate_code("C++ function for computing the n-th Fibonacci number", tokenizer, model))

C++ function for computing the n-th Fibonacci number
 The Fibonacci numbers are the numbers in the following integer sequence:
 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, ...
 The sequence starts with 0 and 1 and each number is the sum of the two preceding ones.
 Write a function in C++ that computes the n-th Fibonacci number.

\begin{code}
#include <iostream>
using namespace std;

int fib
CPU times: user 10.1 s, sys: 122 ms, total: 10.2 s
Wall time: 10.5 s


In [17]:
%%time
print(generate_code("Simple C++ function for computing the n-th Fibonacci number, no description", tokenizer, model))

Simple C++ function for computing the n-th Fibonacci number, no description of the algorithm is given. The function is defined as follows:

\begin{code}
int fib(int n) {
    if (n <= 1) return n;
    return fib(n-1) + fib(n-2);
}
\end{code}

The function is called with the argument 25.

\begin{code}
int main() {
    int n = 25;
    int result = fib(n);
    cout << "fib(" << n << ") = " << result << endl;
   
CPU times: user 10.4 s, sys: 36.7 ms, total: 10.4 s
Wall time: 11.8 s


To test this function in Colab, we first need to write the `.cpp`, then compile it, and only then execute it.

In [26]:
%%writefile test.cpp

#include <iostream>
using namespace std;

int fib(int n) {
    if (n <= 1) return n;
    return fib(n-1) + fib(n-2);
}

int main() {
    int n = 25;
    int result = fib(n);
    cout << "fib(" << n << ") = " << result << endl;
}

Overwriting test.cpp


In [27]:
# Compile
!g++ test.cpp -o test_exe

In [28]:
# Execute
!./test_exe

fib(25) = 75025
